In [21]:
import import_ipynb 
import glob
import pickle
import pandas as pd 
import numpy as np
import time
import csv 

from config import BASE_DIR
from mol2vect import extract_vectorized_molecules, mass_featurize_sdf2vec,molvec_extractor
from sdf_batch_splitting import split_sdf_batch_in_compounds 
from bTree import Node



def start_execution():
    """THIS ROUTINE RUNS ALL THE NEEDED ROUTINES."""

    """BATCH SPLITTING ROUTINE"""
    #split_sdf_batch_in_compounds(25,f'{BASE_DIR}/TEST/BATCH/*.sdf',f'{BASE_DIR}/TEST/SDF_PER_COMPOUND/')  

    """MOL2VEC FEATURIZATION ROUTINE""" 
    #mass_featurize_sdf2vec(SDF_FILES_DIR=f'{BASE_DIR}/TEST/SDF_PER_COMPOUND/',CSV_FILES_DIR = f'{BASE_DIR}/TEST/CSVS/') 

    """MOL2VEC VECTORIZATION"""
    #extract_vectorized_molecules(INPUT_DIR=f'{BASE_DIR}/TEST/CSVS/', OUTPUT_CSV_FILENAME=f'{BASE_DIR}/TEST/vectors_list')
    
# create a dictionary for test compunds
def create_dict_cid_vect():
    cid2vect_dict = {} 

    csvarray = glob.glob(f'{BASE_DIR}/TEST/CSVS/'+'*.csv')

    print('POPULATING  LOOKUP-TABLE:')
    for csvfile in csvarray:
        #get the compound id from the csv filename:
        compid = int((csvfile.split('\\')[-1])[:-4])

        molvec = molvec_extractor(csvfile)
        cid2vect_dict[compid] = list(molvec)
 
    return cid2vect_dict
    
def get_Btree():
    # Get Btree from pkl,in this tree  , every node consist cluster_name [medoid_Cid,[cluster items],parent_cluster_cid,Child_posiotn_of _its_parent]
    # C581 [683, [516, 61, 683], 'C520', 'r'] 
    # C581 cluster has 3 item, its parent cluster is C520 and it is right child of its parent.
    infile = open(BASE_DIR+'/medoid_cluster_tree.pkl', 'rb')
    tree = pickle.load(infile)
    infile.close() 
    return tree

def vect2cid(vect,lookup_table):
    queryres = []

    queryvec = np.asarray(vect)

    for key in lookup_table:
        keyvec = np.asarray((lookup_table[key]))
        if np.array_equal(keyvec, queryvec):
            queryres = key
            break
    return queryres

if __name__ == '__main__':
    start_execution()
    lookup_table = create_dict_cid_vect()
    tree = get_Btree()
   
    # read test dataset from CSV
    dataset = pd.read_csv(f'{BASE_DIR}/TEST/vectors_list.csv',header=None)
    
    startTime = time.time()
    for index in range(len(dataset)): 
        
        new_compound  = test_dataset.loc[index].tolist() # read vector from datasset  
        cid = vect2cid([new_compound],lookup_table)
        print("cid "+ str(cid))
        tree.search_relative_compound(new_compound,1)
        
    executionTime = (time.time() - startTime)
    print('Execution time in seconds: ' + str(executionTime))
    print('Avg Execution time in seconds: ' + str(executionTime/len(dataset)))

POPULATING  LOOKUP-TABLE:
cid 500002
Match found 3
cid 500003
Match found 20
cid 500004
Match found 20
cid 500006
Match found 20
cid 500007
Match found 20
cid 500008
Match found 20
cid 500009
Match found 17
cid 500010
Match found 20
cid 500012
Match found 20
cid 500013
Match found 17
cid 500014
Match found 14
cid 500015
Match found 20
cid 500016
Match found 20
cid 500017
Match found 13
cid 500018
Match found 4
cid 500019
Match found 20
cid 500021
Match found 15
cid 500022
Match found 20
cid 500023
Match found 15
cid 500024
Match found 20
cid 500025
Match found 19
cid 500026
Match found 3
cid 500027
Match found 3
cid 500029
Match found 15
cid 500030
Match found 15
Execution time in seconds: 1.1380467414855957
Avg Execution time in seconds: 0.04552186965942383
